This process was adopted from [Rathje et al](https://www.pnas.org/doi/10.1073/pnas.2308950121)., 2021's open source code. We originally coded this process up in R and used R train GPT on our data. Here is the code translated in Python.

In [ ]:
# install packages

import requests
import pandas as pd
import nltk
import json
import re
import irr
import knitr
import kableExtra
import dplyr

In [ ]:
# load API

my_API = INSERT GPT API KEY

In [ ]:
# write hey chat gpt function to prompt GPT to interpret and then categorize the textual data

def hey_chatGPT(answer_my_question):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {my_API}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-4o-mini",
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": answer_my_question
            }
        ]
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"].strip()

In [ ]:
# load in training dataset

data = pd.read_csv(INSERT TRAINING DATA)
data = data.fillna(0)

In [ ]:
# create new columns for GPT's classifications for each code

data['emojis_gpt'] = None
data['pos_ux_gpt'] = None
data['neg_ux_gpt'] = None
data['pos_ppl_gpt'] = None
data['neg_ppl_gpt'] = None
data['price_gpt'] = None
data['fraud_gpt'] = None
data['ban_gpt'] = None
data['good_match_gpt'] = None
data['bad_match_gpt'] = None
data['pos_sent_gpt'] = None
data['neg_sent_gpt'] = None

In [ ]:
# insert prompt engineering to help GPT understand each code

emojis_question = "Read the following dating app review. Does the review use emojis? Answer only with a number: 1 if the review uses emojis, 0 if the review does not use emojis. Here is the review:"

pos_ux_question = "Read the following dating app review. Does this review praise with the app for working well and feeling genuine? Does the review discuss having a positive user experience (ex. positive experiences with app interface, easy to create a use an account, generally easy and enjoyable to use, enjoying the app features, etc) or any other positive compliments about the UX? A response that simply says the app is good, without expanding on some detail about a good user experience, is not enough to qualify for this positive UX code. Answer only with a number: 1 if the review discusses a the app working well or providing a positive user experience, 0 if the review does not discuss the app working well or providing a positive user experience. Here is the review:"

neg_ux_question = "Read the following dating app review. Does this review discuss any problems with the user experience (ex. ndifficulty logging in and setting up an account, app being slow, having to wait on a waitlist to join the app, unhelpful customer support, running out of likes, egative experiences with the interface, having problems with the app features, etc), or any other negative critiques of the UX? To qualify for the negative UX code, the response needs to name how app feature(s), such as a slow waitlist, led to the overall negative UX. Simply stating complaints about the app, such as being banned or having to pay too much for the account, without naming app feature(s), is not enough to qualify for the negative UX code. Answer only with a number: 1 if the review discusses a negative user experience, 0 if the review does not discuss a negative user experience. Here is the review:"

pos_ppl_question = "Read the following dating app review. Does this review discuss positive social interactions or relationships with other users on the app (ex. making friends, finding love, having great connections and conversations, etc)? Answer only with a number: 1 if the review discusses positive social interactions with other app users, 0 if the review does not discuss positive social interactions with other app users. Here is the review:"

neg_ppl_question = "Read the following dating app review. Does this review discuss negative social interactions or relationships with other users on the app (ex. getting ghosted, being harrassed, not being able to connect with or enjoy conversations, etc)? Answer only with a number: 1 if the review discusses negative social interactions with other app users, 0 if the review does not discuss negative social interactions with other app users. Here is the review:"

price_question = "Read the following dating app review. Does the review discuss any complaints or dissatisfaction with the monetization of the app (ex. the app charges money for too many features, the app's fees and prices are high, the app is wrongfully monetizing love, calling the monetization a scam, etc)? Answer only with a number: 1 if the review discusses negative views about the pricing of the app, 0 if the review does not discuss negative views about the pricing of the app. Here is the review:"

fraud_question = "Read the following dating app review. Does the review discuss encountering fraudulent accounts on the app? To qualify for the fraud code, the review must explicitly be discussing encountering fake dating profiles or inaccurate information about other dating app users. Other discussions of fraud, such as the app being a scam and not being worth the money, does not qualify for this code. Answer only with a number: 1 if the review discusses encountering fraud on the app, 0 if the review does not discuss encountering fraud on the app."

ban_question = "Read the following dating app review. Does the review discuss the reviewr's accout being banned, disabled, shut down, or otherwise closed down so the reviewer cannot access the account? Discussions of the reviewer wanting to cancel their account but being unable to does not qualify for the ban code. Answer only with a number: 1 if the review discusses the reviwer's account being banned, and 0 if the review does not discuss the reviewer's account being banned."

good_match_question = "Read the following dating app review. Does the review discuss the reviewer being happy about the profiles they see on the app (ex. saying they could find love or make friends with the people they see on the app)? Are the recommended matches or other users' dating app profiles that the reviewer encounters on the app as being good and reflective of the types of matches the reviewer was looking for (ex. the app filters showed the reviewer accounts that they were interested in, the potential matches being on the same page as the reviewer, etc)? Simply stating 'good' in the review without further detail does not qualify for this 'good match' code. Answer only with a number: 1 if the review discusses the app providing a good pool of potential matches for the reviewer, 0 if the review does not discuss the app providing a good pool of potential matches for the reviewer."

bad_match_question = "Read the following dating app review. Does the review discuss reiewer having trouble talking to and meeting people on the app? Does the review discuss the recommended matchers being bad (ex. the app filters not working, reviewer doesn't like the matches, the reviewer not finding any matches, etc)? Simply stating 'good' in the review without further detail does not qualify for this 'bad match' code. Stating the app has bugs and has technical difficulties does not qualify for this 'bad match' code. Answer only with a number: 1 if the review discusses the app provides bad potential matches for the reviewer, 0 if the review does not discuss the app providing bad potential matches for the reviewer."

pos_sent_question = "Read the following dating app review. Does the review reflect positive sentiment from the user? Answer only with a number: 1 if the review reflects positive sentiment, 0 if the review does not reflect positive sentiment."

neg_sent_question = "Read the following dating app review. Does the review reflect negative sentiment from the user? Answer only with a number: 1 if the review reflects negative sentiment, 0 if the review does not reflect negative sentiment."

neut_sent_question = "Read the following dating app review. Does the review reflect neutral sentiment from the user? Answer only with a number: 1 if the review reflects neutral sentiment, 0 if the review does not reflect neutral sentiment."

In [ ]:
# write function for GPT to use above prompts to code review text

def process_chatgpt(data, question_variable, output_variable):
    for i in range(len(data)):
        print(i)
        question = data[question_variable][i]
        text = data.iloc[i, 3]
        concat = question + " " + text
        result = hey_chatGPT(concat)
        while not result:
            result = hey_chatGPT(concat)
            print(result)
        print(result)
        data.at[i, output_variable] = result
    return data

In [ ]:
# apply above function for GPT to code review text and insert codes in the created _gpt columns

data = process_chatgpt(data, 'emojis_question', 'emojis_gpt')
data = process_chatgpt(data, 'pos_ux_question', 'pos_ux_gpt')
data = process_chatgpt(data, 'neg_ux_question', 'neg_ux_gpt')
data = process_chatgpt(data, 'pos_ppl_question', 'pos_ppl_gpt')
data = process_chatgpt(data, 'neg_ppl_question', 'neg_ppl_gpt')
data = process_chatgpt(data, 'price_question', 'price_gpt')
data = process_chatgpt(data, 'fraud_question', 'fraud_gpt')
data = process_chatgpt(data, 'ban_question', 'ban_gpt')
data = process_chatgpt(data, 'good_match_question', 'good_match_gpt')
data = process_chatgpt(data, 'bad_match_question', 'bad_match_gpt')
data = process_chatgpt(data, 'pos_sent_question', 'pos_sent_gpt')
data = process_chatgpt(data, 'neg_sent_question', 'neg_sent_gpt')
data = process_chatgpt(data, 'neut_sent_question', 'neut_sent_gpt')
pd.DataFrame(data).head()

In [ ]:
# pre process data

cols = ["emojis_gpt", "pos_ux_gpt", "neg_ux_gpt", "pos_ppl_gpt", "neg_ppl_gpt", "price_gpt", "fraud_gpt", "ban_gpt", "good_match_gpt", "bad_match_gpt", "pos_sent_gpt", "neg_sent_gpt", "neut_sent_gpt"]
og_cols = ["emojis", "pos_ux", "neg_ux", "pos_ppl", "neg_ppl", "price", "fraud", "ban", "good_match", "bad_match", "pos_sent", "neg_sent", "neut_sent"]

for col in cols:
    data[col] = data[col].str[:1]

for og_col in og_cols:
    data[og_col] = data[og_col].str[:1]

for col in cols:
    data[col] = pd.to_numeric(data[col])

for og_col in og_cols:
    data[og_col] = pd.to_numeric(data[og_col])

In [ ]:
# run correlation table to test interrater reliability between training data and GPT
from scipy.stats import pearsonr

# Define a list of variable pairs
pairs = {
    "emojis": ["emojis", "emojis_gpt"],
    "pos_ux": ["pos_ux", "pos_ux_gpt"],
    "neg_ux": ["neg_ux", "neg_ux_gpt"],
    "price": ["price", "price_gpt"],
    "fraud": ["fraud", "fraud_gpt"],
    "ban": ["ban", "ban_gpt"],
    "good_match": ["good_match", "good_match_gpt"],
    "bad_match": ["bad_match", "bad_match_gpt"],
    "pos_sent": ["pos_sent", "pos_sent_gpt"],
    "neg_sent": ["neg_sent", "neg_sent_gpt"],
    "neut_sent": ["neut_sent", "neut_sent_gpt"]
}

# Create an empty data frame to store results
corr_results = pd.DataFrame(columns=["variable", "correlation"])

# Loop over each pair, run the correlation test, and extract the relevant info
for var, cols in pairs.items():
    col1, col2 = cols

    # Run the correlation test
    test_result = pearsonr(data[col1], data[col2])

    # Append results to the data frame
    corr_results = corr_results.append({"variable": var, "correlation": test_result[0]}, ignore_index=True)

# Print the results
print(corr_results)

In [ ]:
# run kappa values to test interrater reliability between training data and GPT

import pandas as pd
from statsmodels.stats.inter_rater import kappa2

# Create an empty data frame to store the results
kappa_results = pd.DataFrame(columns=['variable', 'kappa'])

# Loop over each pair, calculate kappa2, and extract the value and p-value
for var in pairs:
    test_result = kappa2(data[pairs[var]], weight='unweighted')
    kappa_val = test_result.value
    kappa_results = kappa_results.append({'variable': var, 'kappa': kappa_val}, ignore_index=True)

# Print the results table
print(kappa_results)

In [ ]:
# pull specific codes out and see where GPT and trianing data have mismatches. Can repeat for every code.

from kaleido.scopes.plotly import PlotlyScope

data = pd.read_csv('data.csv')
data = data[data['pos_ux'] != data['pos_ux_gpt']]
data = data[['content', 'pos_ux', 'pos_ux_gpt']]
data.to_html('output.html', index=False)

We can then repeat this process for the dataset we want to analyze, just by swapping out the dataframe and removing the interrater reliability checking steps!